In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import json

#### Get data and preparation

In [2]:
# get token
import requests

## access burning glass and get token
BURNING_GLASS_USERNAME = "dhiraj.sharda@citi.com"
BURNING_GLASS_PASSWORD = "BurningGlass123$"

BURNING_GLASS_DOMAIN = "https://apis.burning-glass.com/"
BURNING_GLASS_FETCH_TOKEN_PATH = "auth/realms/BGT/protocol/openid-connect/token"
BURNING_GLASS_VERSION = "v3.4"
base_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_VERSION
token_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_FETCH_TOKEN_PATH

def get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD):
    params = {'grant_type': 'password',
                'client_id': 'UnifiedApi',
                'username': BURNING_GLASS_USERNAME,
                'password': BURNING_GLASS_PASSWORD}
    
    response = requests.request("POST", token_url, data=params)

    access_token = response.json()['access_token']
    expires = response.json()['expires_in']
    
    headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(access_token)}
    
    return access_token, expires, headers
    
# run this when token expires
# you need the access token to make any calls to the API
# it expires after about 24hrs so you just need to run this code again to get a new one
access_token, expires, header = get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD)

In [3]:
# download from api
# /v3.5/ontology/suboccs
import requests

## access burning glass and get token
BURNING_GLASS_USERNAME = "dhiraj.sharda@citi.com"
BURNING_GLASS_PASSWORD = "BurningGlass123$"

BURNING_GLASS_DOMAIN = "https://apis.burning-glass.com/"
BURNING_GLASS_FETCH_TOKEN_PATH = "auth/realms/BGT/protocol/openid-connect/token"
BURNING_GLASS_VERSION = "v3.4"
base_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_VERSION
token_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_FETCH_TOKEN_PATH

def get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD):
    params = {'grant_type': 'password',
                'client_id': 'UnifiedApi',
                'username': BURNING_GLASS_USERNAME,
                'password': BURNING_GLASS_PASSWORD}
    
    response = requests.request("POST", token_url, data=params)

    access_token = response.json()['access_token']
    expires = response.json()['expires_in']
    
    headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(access_token)}
    
    return access_token, expires, headers
import http.client


# payload = "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX bgt: <http://www.burning-glass.com/Ontology#> SELECT ?label WHERE { ?class a bgt:BaselineSkill. ?class rdfs:label ?label FILTER langMatches(lang(?label), \"en-us\") } LIMIT 25"

access_token, expires, header = get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD)
# headers = {
#     'accept': "application/json",
#     'authorization': "Bearer YOUR ACCESS TOKEN HERE",
#     'accept-language': "en-us"
#     }


# This is the difference-----start
conn = http.client.HTTPSConnection("apis.burning-glass.com")
conn.request("GET", "/v3.5/ontology/suboccs", headers=header)
# conn.request("POST", "/v3.5/ontology/query", payload, header)

# End


res = conn.getresponse()
data = res.read()

#print(data.decode("utf-8"))
data_str = data.decode("utf-8")

In [4]:
# get all jobs
data_str = data.decode("utf-8")
label_list = [i.start() for i in re.finditer('label', data_str)]
label_to = []
for i in range(len(label_list)):
    a_temp = label_list[i]
    b_temp = data_str[a_temp+8:a_temp+300].index('",')
    c_temp = data_str[a_temp+8:a_temp+b_temp+8]
    label_to.append(c_temp)

In [5]:

import requests

## access burning glass and get token
BURNING_GLASS_USERNAME = "dhiraj.sharda@citi.com"
BURNING_GLASS_PASSWORD = "BurningGlass123$"

BURNING_GLASS_DOMAIN = "https://apis.burning-glass.com/"
BURNING_GLASS_FETCH_TOKEN_PATH = "auth/realms/BGT/protocol/openid-connect/token"
BURNING_GLASS_VERSION = "v3.4"
base_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_VERSION
token_url = BURNING_GLASS_DOMAIN + BURNING_GLASS_FETCH_TOKEN_PATH

def get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD):
    params = {'grant_type': 'password',
                'client_id': 'UnifiedApi',
                'username': BURNING_GLASS_USERNAME,
                'password': BURNING_GLASS_PASSWORD}
    
    response = requests.request("POST", token_url, data=params)

    access_token = response.json()['access_token']
    expires = response.json()['expires_in']
    
    headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(access_token)}
    
    return access_token, expires, headers
    
# run this when token expires
# you need the access token to make any calls to the API
# it expires after about 24hrs so you just need to run this code again to get a new one
access_token, expires, header = get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD)

print(access_token)

# use 
def make_call(add_url, payload, type_='GET', base_url=base_url, headers=header, print_url=False):
    global access_token
    global header
    url = base_url + add_url
    if print_url:
        print(url)
    response = requests.request(type_, url, data=payload, headers=header).json()
    
    if "error" in response:
        if response["error"]["message"] == "Unauthorized":
            print('getting new token')   

            access_token, _, header = get_token(token_url, BURNING_GLASS_USERNAME, BURNING_GLASS_PASSWORD)
            response = requests.request(type_, url, data=payload, headers=header).json()
        else:
            print("error in API call", response["error"]["message"])

    return response

def get_subocc_info(roles, msa=None):
    add_url = "/profile/subocc"
    # needs to be in list format
    if isinstance(roles, str):
        roles = [roles]
    
    if msa == None:
        payload = '{"name": %s}' %roles
        payload = payload.replace("'",'"')
    else:
        payload = '{"name": %s, "geography": {"msa": "%s"}}' %(roles, msa)
        payload = payload.replace("'",'"')

    response = make_call(add_url, payload=payload, type_='POST', print_url=True)     
#    print(response)
    return response['response']['data']

# role_list = ['Data Scientist', 'Data Engineer',label_to[2]]
# role_list2 = label_to[0:80]
# response = get_subocc_info(role_list2)
# response
response_list = []
for i in label_to[0:1000]:
    try:
        role_list2 = i
        response = get_subocc_info([role_list2])
        response_list.append(response)
    except:
        continue
    

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJTQnFKdmwzZ3dSWTBnaERob0pGam5pN0hhN2x6ZDRpZTVmOHFOMkJkZ3dZIn0.eyJqdGkiOiIwODcwZTQ1Yy05YTA1LTQ5YjItYWRhNS1lNGU2OTQ2NDNkMTgiLCJleHAiOjE1ODU2ODkxMTAsIm5iZiI6MCwiaWF0IjoxNTg1NTE2MzEwLCJpc3MiOiJodHRwczovL2FwaXMuYnVybmluZy1nbGFzcy5jb20vYXV0aC9yZWFsbXMvQkdUIiwiYXVkIjoiVW5pZmllZEFwaSIsInN1YiI6IjRkYjI2M2MzLTdlYzUtNDdmZC1hMDcxLTliMzdmNDY2ZTk0ZSIsInR5cCI6IkJlYXJlciIsImF6cCI6IlVuaWZpZWRBcGkiLCJhdXRoX3RpbWUiOjAsInNlc3Npb25fc3RhdGUiOiJiOWJlNTExMS01Mjc3LTQ3OTctYjIyZS0xMzA4YzkyZjhmNWYiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIioiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiVW5pZmllZEFwaSI6eyJyb2xlcyI6WyJBUEk6IE9jY3VwYXRpb24gUHJvZmlsZSBBUEkiLCJVc2UgQ2FzZTogUm9sZSBQcmljaW5nIiwiVXNlIENhc2U6IE9jY3VwYXRpb24gRXhwbG9yYXRpb24iLCJBUEk6IFNhbGFyeSBBUEkiLCJBUEk6IE9udG9sb2d5IEFQSSIsIkFQSTogU2VhcmNoIEFQSSIsIkFQSTogT2NjdXBhdGlvbiBDbGFzc2lmaWVyIEFQSSIsIkFQSTogU2ltaWxhcml0eSBBUEkiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFj

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/v3.4/profile/subocc
https://apis.burning-glass.com/

In [93]:
# save data
import json
fileObject = open('citidata.json', 'w')
jsObj = json.dumps(response_list)
fileObject.write(jsObj) 
fileObject.close()

In [94]:
# read data
with open('citidata.json', 'r') as f:
    data_json = json.load(f)